In [5]:
Item.objects.filter(pk=401).update(list_thumbnail='items/thumnail_401.jpg')

1

In [33]:
suffix = 1085
for pk in range(1085, 1092):
    Item.objects.filter(pk=pk).update(list_thumbnail='items/thumnail_'+str(suffix)+'.jpg')
    suffix += 1
    print(pk)

1085
1086
1087
1088
1089
1090
1091


In [20]:
Item.objects.filter(pk=924).update(list_thumbnail='')

1

In [19]:
Item.objects.filter(pk=839).update(list_thumbnail='items/thumnail_'+str(841)+'.jpg')

1

In [23]:
Item.objects.filter(pk=924).update(list_thumbnail='')

1

In [24]:
Item.objects.filter(pk=925).update(list_thumbnail='')

1

In [25]:
Item.objects.filter(pk=926).update(list_thumbnail='')

1

In [26]:
Item.objects.filter(pk=927).update(list_thumbnail='')

1

In [29]:
Item.objects.filter(pk=976).update(list_thumbnail='')

1

In [1]:
col_detail_url = [
    'https://www.baeminchan.com/mealplan/detail.php?hash=G0F42&cno=46110000',
    'https://www.baeminchan.com/mealplan/detail.php?hash=F1256&cno=46110000',
    'https://www.baeminchan.com/mealplan/detail.php?hash=F23AA&cno=46110000',
    'https://www.baeminchan.com/mealplan/detail.php?hash=F62EE&cno=46110000',
    'https://www.baeminchan.com/mealplan/detail.php?hash=FF7CB&cno=46110000',
    'https://www.baeminchan.com/mealplan/detail.php?hash=G27B7&cno=46110000',
    'https://www.baeminchan.com/mealplan/detail.php?hash=FDAEB&cno=46110000',
    'https://www.baeminchan.com/mealplan/detail.php?hash=GAF8D&cno=46110000',
    'https://www.baeminchan.com/mealplan/detail.php?hash=GA3FA&cno=46110000',
    'https://www.baeminchan.com/mealplan/detail.php?hash=GCA07&cno=46110000',
    'https://www.baeminchan.com/mealplan/detail.php?hash=F6B07&cno=46110000',    
]

In [2]:
from bs4 import BeautifulSoup
import requests, re

url = 'https://www.baeminchan.com/mealplan/list.php'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'lxml')
soup_products = soup.find('ul', id='products')
col_products = []
col_products = soup_products.findAll('li')
col_ga_id = []
for col in col_products:
    col_ga_id.append(col.select('a')[0].get('ga_id'))
  

In [39]:
for col in col_ga_id:
    print(col)

10815
5825
5826
1890
5815
9915
5881
12481
12841
6934
5648


In [4]:
for url in col_detail_url:
    print(url)
    ga_id = col_ga_id[col_detail_url.index(url)]
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'lxml')
    soup_detail = soup.find('div', id='app')
    
    added_words = ''
    try:
        added_words = soup_detail.find_all("p", "desc_bt_txt")[0].get_text()
    except:
        pass
            
    point = 0
    try:
        int(soup_detail.find_all("dl", "desc_info")[0].select('dd')[0].get_text().replace('원',''))
    except:
        pass
    else:
        point = int(soup_detail.find_all("dl", "desc_info")[0].select('dd')[0].get_text().replace('원',''))
            
    delivery_type = ''
    try:        
        delivery_type = soup_detail.find_all("dl", "desc_info")[0].select('dd')[1].get_text().replace('\r','').replace('\n','')
    except:
        pass                    
    
    receive_day = ''
    try:
        receive_day = soup_detail.find_all("dl", "desc_info")[0].select('dd')[2].get_text().replace('\r','').replace('\n','')
    except:
        pass
    
    regular_delivery = ''
    try:
        regular_delivery = soup_detail.find_all("dl", "desc_info")[0].select('dd')[3].get_text()
    except:
        pass
    
    if regular_delivery == '가능':
        regular_delivery = True
    else:
        regular_delivery = False
        
    soup_detail_description = soup.find('div', id='detail_official_information')
    col_is_equal = [
        '식품의 유형',
        '생산자 및 소재지(수입품의 경우 수입자를 함께 표기)',
        '제조연월일, 유통기한 또는 품질유지기한',
        '포장단위별 용량(중량), 수량',
        '원재료명 및 함량',
        '영양성분',
        '유전자재조합식품 여부',
        '영유아식 또는 체중조절식품 등에 해당하는 경우',
        '수입식품 문구',
        '소비자상담 관련 전화번호',    
        '알레르기 주의사항',
        '알레르기 유발물질',
        '품목신고번호',
    ]
    col_th = []
    col_td = []
    
    col_th = soup_detail_description.find_all('tbody')[0].select('th')
    col_td = soup_detail_description.find_all('tbody')[0].select('td')
    
    col_description_data = []

    for col in col_is_equal:
        descripiton = ''
        for col_th_num in col_th:
            if col == col_th_num.get_text():
                descripiton = col_td[col_th.index(col_th_num)].get_text().replace('\r','').replace('\n','')
                break

        col_description_data.append(descripiton)
    print(added_words)
    Description.objects.create(
        item=Item.objects.get(ga_id=ga_id), 
        added_words=added_words,
        point=point,
        delivery_type=delivery_type,
        receive_day=receive_day,
        regular_delivery=regular_delivery,
        item_type=col_description_data[0],
        factory_address=col_description_data[1],
        dom=col_description_data[2],
        capacity=col_description_data[3],
        ingredient=col_description_data[4],
        caution=col_description_data[10],                
        allergy_material=col_description_data[11],        
    )
    
#     print(point)
#     print(delivery_type)
#     print(receive_day)
#     print(regular_delivery)
#     print(col_description_data[0])
#     print(col_description_data[1])
#     print(col_description_data[2])
#     print(col_description_data[3])
#     print(col_description_data[4])
#     print(col_description_data[10])
#     print(col_description_data[11])
#     Description.objects.create(
#             item=Item.objects.get(ga_id=ga_id), 
#             added_words=added_words,
#             point=point,
#             delivery_type=delivery_type,
#             receive_day=receive_day,
#             regular_delivery=regular_delivery,
#             item_type=col_description_data[0],
#             factory_address=col_description_data[1],
#             dom=col_description_data[2],
#             capacity=col_description_data[3],
#             ingredient=col_description_data[4],
#             caution=col_description_data[10],                
#             allergy_material=col_description_data[11],        
#         )

    try:
        Description.objects.create(
            item=Item.objects.get(ga_id=ga_id), 
            added_words=added_words,
            point=point,
            delivery_type=delivery_type,
            receive_day=receive_day,
            regular_delivery=regular_delivery,
            item_type=col_description_data[0],
            factory_address=col_description_data[1],
            dom=col_description_data[2],
            capacity=col_description_data[3],
            ingredient=col_description_data[4],
            caution=col_description_data[10],                
            allergy_material=col_description_data[11],        
        )
        print('success')
    except:
        pass   

https://www.baeminchan.com/mealplan/detail.php?hash=G0F42&cno=46110000
국내 최고 요리연구가 방영아의 제안


DataError: value too long for type character varying(100)


In [46]:
item = Item.objects.get(ga_id='5648')

In [47]:
item.id

1080

In [25]:
item = Description.objects.get(item=Item.objects.get(pk=1070))

In [26]:
item

<Description: [Young313]아이반찬(반찬4,국1가지)>

In [11]:
Description.objects.filter(item=Item.objects.get(pk=1080)).delete()

(1, {'items.Description': 1})